In [1]:
from dotenv import load_dotenv
import os 

load_dotenv()

True

In [2]:
import pandas as pd

In [2]:
from snowflake.snowpark import Session

In [3]:
connection_parameters = {
    "account": os.environ["snowflake_account"],
    "user": os.environ["snowflake_user"],
    "password": os.environ["snowflake_password"],
    "role": os.environ["snowflake_user_role"],
    "warehouse": os.environ["snowflake_warehouse"],
    "database": os.environ["snowflake_database"],
    "schema": os.environ["snowflake_schema"]
}

session = Session.builder.configs(connection_parameters).create()

In [4]:
session.sql('SELECT * FROM LEAD_SCORING LIMIT 10').collect()

[Row(PROSPECT_ID='7927b2df-8bba-4d29-b9a2-b6e0beafe620', LEAD_NUMBER=660737, LEAD_ORIGIN='API', LEAD_SOURCE='Olark Chat', DO_NOT_EMAIL=False, DO_NOT_CALL=False, CONVERTED=0, TOTALVISITS=0, TOTAL_TIME_SPENT_ON_WEBSITE=0, PAGE_VIEWS_PER_VISIT=0.0, LAST_ACTIVITY='Page Visited on Website', COUNTRY=None, SPECIALIZATION='Select', HOW_DID_YOU_HEAR_ABOUT_X_EDUCATION='Select', WHAT_IS_YOUR_CURRENT_OCCUPATION='Unemployed', WHAT_MATTERS_MOST_TO_YOU_IN_CHOOSING_A_COURSE='Better Career Prospects', SEARCH=False, MAGAZINE=False, NEWSPAPER_ARTICLE=False, X_EDUCATION_FORUMS=False, NEWSPAPER=False, DIGITAL_ADVERTISEMENT=False, THROUGH_RECOMMENDATIONS=False, RECEIVE_MORE_UPDATES_ABOUT_OUR_COURSES=False, TAGS='Interested in other courses', LEAD_QUALITY='Low in Relevance', UPDATE_ME_ON_SUPPLY_CHAIN_CONTENT=False, GET_UPDATES_ON_DM_CONTENT=False, LEAD_PROFILE='Select', CITY='Select', ASYMMETRIQUE_ACTIVITY_INDEX='02.Medium', ASYMMETRIQUE_PROFILE_INDEX='02.Medium', ASYMMETRIQUE_ACTIVITY_SCORE=15, ASYMMETRIQUE

In [5]:
leads = session.table("LEAD_SCORING")

In [6]:
type(leads)

snowflake.snowpark.table.Table

In [7]:
leads.show()

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"PROSPEC

In [9]:
df = leads.to_pandas()

In [10]:
df.head()

,PROSPECT_ID,LEAD_NUMBER,LEAD_ORIGIN,LEAD_SOURCE,DO_NOT_EMAIL,DO_NOT_CALL,CONVERTED,TOTALVISITS,TOTAL_TIME_SPENT_ON_WEBSITE,PAGE_VIEWS_PER_VISIT,...,GET_UPDATES_ON_DM_CONTENT,LEAD_PROFILE,CITY,ASYMMETRIQUE_ACTIVITY_INDEX,ASYMMETRIQUE_PROFILE_INDEX,ASYMMETRIQUE_ACTIVITY_SCORE,ASYMMETRIQUE_PROFILE_SCORE,I_AGREE_TO_PAY_THE_AMOUNT_THROUGH_CHEQUE,A_FREE_COPY_OF_MASTERING_THE_INTERVIEW,LAST_NOTABLE_ACTIVITY
0,7927b2df-8bba-4d29-b9a2-b6e0beafe620,660737,API,Olark Chat,False,False,0,0.0,0,0.0,...,False,Select,Select,02.Medium,02.Medium,15.0,15.0,False,False,Modified
1,2a272436-5132-4136-86fa-dcc88c88f482,660728,API,Organic Search,False,False,0,5.0,674,2.5,...,False,Select,Select,02.Medium,02.Medium,15.0,15.0,False,False,Email Opened
2,8cc8c611-a219-4f35-ad23-fdfd2656bd8a,660727,Landing Page Submission,Direct Traffic,False,False,1,2.0,1532,2.0,...,False,Potential Lead,Mumbai,02.Medium,01.High,14.0,20.0,False,True,Email Opened
3,0cc2df48-7cf4-4e39-9de9-19797f9b38cc,660719,Landing Page Submission,Direct Traffic,False,False,0,1.0,305,1.0,...,False,Select,Mumbai,02.Medium,01.High,13.0,17.0,False,False,Modified
4,3256f628-e534-4826-9d63-4a8b88782852,660681,Landing Page Submission,Google,False,False,1,2.0,1428,1.0,...,False,Select,Mumbai,02.Medium,01.High,15.0,18.0,False,False,Modified


In [11]:
len(df)

9240

In [12]:
df['TOTAL_TIME_SPENT_ON_WEBSITE'].describe()

count    9240.000000
mean      487.698268
std       548.021466
min         0.000000
25%        12.000000
50%       248.000000
75%       936.000000
max      2272.000000
Name: TOTAL_TIME_SPENT_ON_WEBSITE, dtype: float64

In [13]:
quantile_list = [0, .25, .5, .75, 1.]
quantiles = df['TOTAL_TIME_SPENT_ON_WEBSITE'].quantile(quantile_list)
quantiles

0.00       0.0
0.25      12.0
0.50     248.0
0.75     936.0
1.00    2272.0
Name: TOTAL_TIME_SPENT_ON_WEBSITE, dtype: float64

In [17]:
df['TIME_ON_WEBSITE_QUANTILE_RANGE'] = pd.qcut(
                                            df['TOTAL_TIME_SPENT_ON_WEBSITE'], 
                                            q=quantile_list)

In [19]:
quantile_labels = ['1Q', '2Q', '3Q', '4Q']
df['TIME_ON_WEBSITE_QUANTILE_LABELS'] = pd.qcut(
                                            df['TOTAL_TIME_SPENT_ON_WEBSITE'], 
                                            q=quantile_list,       
                                            labels=quantile_labels)

In [20]:
df[['TOTAL_TIME_SPENT_ON_WEBSITE','TIME_ON_WEBSITE_QUANTILE_RANGE','TIME_ON_WEBSITE_QUANTILE_LABELS']].head()

,TOTAL_TIME_SPENT_ON_WEBSITE,TIME_ON_WEBSITE_QUANTILE_RANGE,TIME_ON_WEBSITE_QUANTILE_LABELS
0,0,"(-0.001, 12.0]",1Q
1,674,"(248.0, 936.0]",3Q
2,1532,"(936.0, 2272.0]",4Q
3,305,"(248.0, 936.0]",3Q
4,1428,"(936.0, 2272.0]",4Q


In [27]:
df = df.drop('TIME_ON_WEBSITE_QUANTILE_RANGE', axis=1)

In [29]:
df[['TOTAL_TIME_SPENT_ON_WEBSITE','TIME_ON_WEBSITE_QUANTILE_LABELS']].head()

,TOTAL_TIME_SPENT_ON_WEBSITE,TIME_ON_WEBSITE_QUANTILE_LABELS
0,0,1Q
1,674,3Q
2,1532,4Q
3,305,3Q
4,1428,4Q


In [30]:
#df.write.mode("overwrite").save_as_table("LEAD_SCORING")

In [31]:
#session.create_dataframe?

In [33]:
foo_tbl = session.create_dataframe(df)

In [35]:
foo_tbl.write.mode("overwrite").save_as_table("LEAD_SCORING_Q")

In [36]:
new_leads = session.table("LEAD_SCORING_Q")

In [40]:
new_leads[new_leads['TIME_ON_WEBSITE_QUANTILE_LABELS']]

In [41]:
from snowflake.snowpark.functions import col

In [42]:
result = session.table("LEAD_SCORING_Q").filter(col("TIME_ON_WEBSITE_QUANTILE_LABELS") == '1Q')

In [44]:
result_df = result.to_pandas()
result_df.head()

,PROSPECT_ID,LEAD_NUMBER,LEAD_ORIGIN,LEAD_SOURCE,DO_NOT_EMAIL,DO_NOT_CALL,CONVERTED,TOTALVISITS,TOTAL_TIME_SPENT_ON_WEBSITE,PAGE_VIEWS_PER_VISIT,...,LEAD_PROFILE,CITY,ASYMMETRIQUE_ACTIVITY_INDEX,ASYMMETRIQUE_PROFILE_INDEX,ASYMMETRIQUE_ACTIVITY_SCORE,ASYMMETRIQUE_PROFILE_SCORE,I_AGREE_TO_PAY_THE_AMOUNT_THROUGH_CHEQUE,A_FREE_COPY_OF_MASTERING_THE_INTERVIEW,LAST_NOTABLE_ACTIVITY,TIME_ON_WEBSITE_QUANTILE_LABELS
0,7927b2df-8bba-4d29-b9a2-b6e0beafe620,660737,API,Olark Chat,False,False,0,0.0,0,0.0,...,Select,Select,02.Medium,02.Medium,15.0,15.0,False,False,Modified,1Q
1,2058ef08-2858-443e-a01f-a9237db2f5ce,660680,API,Olark Chat,False,False,0,0.0,0,0.0,...,None,None,01.High,02.Medium,17.0,15.0,False,False,Modified,1Q
2,20ef72a2-fb3b-45e0-924e-551c5fa59095,660664,API,Olark Chat,False,False,0,0.0,0,0.0,...,None,None,02.Medium,02.Medium,15.0,15.0,False,False,Modified,1Q
3,3abb7c77-1634-4083-9a9f-861068220611,660540,API,Olark Chat,False,False,0,0.0,0,0.0,...,None,None,01.High,02.Medium,17.0,15.0,False,False,Modified,1Q
4,3f04fd93-2884-45f6-a4fc-f3b6f234554b,660153,API,Olark Chat,False,False,0,0.0,0,0.0,...,None,None,01.High,02.Medium,17.0,15.0,False,False,Modified,1Q
